# Markov chains

This notebook contains code examples referring to the Markov chains chapter of "Applied Mathematics with Open-Source Software: Operational Research Problems
with Python and R".

First we define a function to get the transition rates between two given states:

In [2]:
def get_transition_rate( in_state,
    out_state,
    waiting_room=4,
    num_barbers=2,
):
    """Return the transition rate for 2 given states.
    Args:
        in_state: an integer
        out_state: an integer
        waiting_room: an integer (default: 4)
        num_barbers:  an integer (default: 2)
    Returns:
        A real.
    """
    arrival_rate = 10
    service_rate = 4
    capacity = waiting_room + num_barbers
    delta = out_state - in_state
    if delta == 1 and in_state < capacity: 
        return arrival_rate
    if delta == -1:
        return min(in_state, num_barbers) * service_rate
    return 0

This can be used to create the transition matrix:

In [5]:
import itertools
import numpy as np


def get_transition_rate_matrix(waiting_room=4, num_barbers=2):
    """Return the transition matrix Q.

    Args:
        waiting_room: an integer (default: 4)
        num_barbers: an integer (default: 2)

    Returns:
        A matrix.
    """
    capacity = waiting_room + num_barbers
    state_pairs = itertools.product(range(capacity + 1), repeat=2)
    flat_transition_rates = [
        get_transition_rate(
            in_state=in_state,
            out_state=out_state,
            waiting_room=waiting_room,
            num_barbers=num_barbers,
        )
        for in_state, out_state in state_pairs
    ]
    transition_rates = np.reshape(
        flat_transition_rates, (capacity + 1, capacity + 1)
    )
    np.fill_diagonal(
        transition_rates, -transition_rates.sum(axis=1)
    )

    return transition_rates

For the problem from the chapter this is:

In [7]:
Q = get_transition_rate_matrix()
print(Q)

[[-10  10   0   0   0   0   0]
 [  4 -14  10   0   0   0   0]
 [  0   8 -18  10   0   0   0]
 [  0   0   8 -18  10   0   0]
 [  0   0   0   8 -18  10   0]
 [  0   0   0   0   8 -18  10]
 [  0   0   0   0   0   8  -8]]


Here is the state of the system after 0.5 time units:

In [8]:
import scipy.linalg

print(scipy.linalg.expm(Q * 0.5).round(5))

[[0.10492 0.21254 0.20377 0.17142 0.13021 0.09564 0.0815 ]
 [0.08501 0.18292 0.18666 0.1708  0.14377 0.1189  0.11194]
 [0.06521 0.14933 0.16338 0.16478 0.15633 0.14751 0.15346]
 [0.04388 0.10931 0.13183 0.15181 0.16777 0.18398 0.21142]
 [0.02667 0.07361 0.10005 0.13422 0.17393 0.2189  0.27262]
 [0.01567 0.0487  0.07552 0.11775 0.17512 0.24484 0.32239]
 [0.01068 0.03668 0.06286 0.10824 0.17448 0.25791 0.34914]]


After 500 time units:

In [9]:
print(scipy.linalg.expm(Q * 500).round(5))

[[0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]
 [0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]
 [0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]
 [0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]
 [0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]
 [0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]
 [0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]]


We can obtain the steady state:

In [11]:
def get_steady_state_vector(Q):
    """Return the steady state vector of any given continuous time
    transition rate matrix.

    Args:
       Q: a transition rate matrix

    Returns:
        A vector
    """
    state_space_size, _ = Q.shape
    A = np.vstack((Q.T, np.ones(state_space_size)))
    b = np.append(np.zeros(state_space_size), 1)
    x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    return x

In [12]:
print(get_steady_state_vector(Q).round(5))

[0.03431 0.08577 0.10722 0.13402 0.16752 0.2094  0.26176]


The probability of a full shop

In [14]:
def get_probability_of_full_shop(waiting_room=4, num_barbers=2):
    """Return the probability of the barber shop being full.

    Args:
        waiting_room: an integer (default: 4)
        num_barbers: an integer (default: 2)

    Returns:
        A real.
    """
    Q = get_transition_rate_matrix(
        waiting_room=waiting_room,
        num_barbers=num_barbers,
    )
    pi = get_steady_state_vector(Q)
    return pi[-1]

In [15]:
print(round(get_probability_of_full_shop(), 6))

0.261756


In [16]:
print(round(get_probability_of_full_shop(waiting_room=6), 6))

0.23557


In [17]:
print(round(get_probability_of_full_shop(num_barbers=3), 6))

0.078636
